We start by defining a couple of variables which will be necessary to run the code in this notebook. 
The `model_params` data frame will contain data that is to be set by the user. Variables outside of the data frame are needed for the functions to run, but will be later defined somewhere else. 

In [1]:
model_params <- data.frame(
  # age at baseline
  n_age_init = 1,
  # maximum age of follow up
  n_age_max  = 8,
  # discount rate for costs and QALYS 
  d_r = 0.04,
  # number of simulations
  n_sim   = 1000
)

n_t <- model_params$n_age_max - model_params$n_age_init

model_params$n_t <- n_t

# the 4 health states of the model:
v_n <- c("S2", "S3", "D") 

# number of health states 
n_states <- length(v_n) 


This is a data frame containing more parameters that are to be set by the user. Specifically, these parameters relate to probabilistic sensitivity analysis (PSA). They define a probability distribution that will be used to randomly generate data for each parameter, a given number of times. Parameters in health economics models must be defined and duly justified according to real world evidence (RWE), commonly from sources such as randomized controlled trials (RCTs). Using PSA allows us to account for uncertainty in our data, increasing confidence in our results.

As of january 12th, 2025, the data set here are arbitrary, simply to demonstrate that the model works. Later, it will be replaced by duly justified data. 

In [2]:
library(truncnorm)

default_psa_params <- list(
  # prob Stage 2 -> Stage 3 if untreated
  p_S2_S3_unt = list(
    name = "P(S2 → S3) if untreated",
    def_dis = "beta", 
    def_v1 = "30", 
    def_v2 = "170"
  ),
  
  # rate ratio Stage 2 -> Stage 3 treated/untreated
  hr_S2_S3 = list(
    name = "Hazard Ratio for P(S2 → S3) Treated/Untreated",
    def_dis = "normal", 
    def_v1 = "0.7", 
    def_v2 = "0.3"
  ),
  
  # prob Stage 2 -> Dead
  p_S2_D = list(
    name = "P(S2 → Dead)",
    def_dis = "beta", 
    def_v1 = "1", 
    def_v2 = "1990"
  ),
  
  # rate ratio death Stage 3 vs Stage 2
  hr_S3_D = list(
    name = "Hazard Ratio for death S3/S2",
    def_dis = "normal", 
    def_v1 = "0.1", 
    def_v2 = "0.03"
  ),
  
  # cost p/cycle in state S2
  c_S2 = list(
    name = "Cost per cycle in Stage 2",
    def_dis = "gamma", 
    def_v1 = "100", 
    def_v2 = "20"
  ),
  
  # cost p/cycle in state S3
  c_S3 = list(
    name = "Cost per cycle in Stage 3",
    def_dis = "gamma", 
    def_v1 = "177.8", 
    def_v2 = "22.5"
  ),
  
  # cost p/cycle of treatment in state S2
  c_Trt_S2 = list(
    name = "Cost of treatment in Stage 2",
    def_dis = "fixed", 
    def_v1 = "200", 
    def_v2 = ""
  ),
  
  # cost p/cycle of treatment in state S3 (we stop treating in S3)
  c_Trt_S3 = list(
    name = "Cost of treatment in Stage 3",
    def_dis = "fixed", 
    def_v1 = "0", 
    def_v2 = ""
  ),

  # cost p/cycle in state D (fixed value)
  c_D = list(
    name = "Cost of Death",
    def_dis = "fixed", 
    def_v1 = "0", 
    def_v2 = ""
  ),
  
  # utility when stage 2
  u_S2 = list(
    name = "Utility in Stage 2",
    def_dis = "normal", 
    def_v1 = "1", 
    def_v2 = "0.01"
  ),
  
  # utility when stage 3
  u_S3 = list(
    name = "Utility in Stage 3",
    def_dis = "normal", 
    def_v1 = "0.75", 
    def_v2 = "0.02"
  ),
  
  # utility when dead
  u_D = list(
    name = "Utility when dead",
    def_dis = "fixed", 
    def_v1 = "0", 
    def_v2 = ""
  )
)


We start by generating a data frame of parameters according to our probability distribution, containing `n-sim` rows (`n-sim` being a property of the previously set `model_params` data frame) : this corresponds to the number of times our model is to be run, currently set to 1000.

In [3]:
f_gen_psa <- function(psa_params, model_params) {
  # Initialize an empty list to store the generated data
  df_list <- list()
  
  # Iterate over each parameter in the psa_params list
  for (param_name in names(psa_params)) {
    param_info <- psa_params[[param_name]]
    dist_type <- param_info$def_dis
    v1 <- as.numeric(param_info$def_v1)
    v2 <- as.numeric(param_info$def_v2)
    
    # Generate samples based on the distribution type
    samples <- switch(dist_type,
      "beta" = rbeta(n = model_params$n_sim, shape1 = v1, shape2 = v2),
      "log_normal" = rlnorm(n = model_params$n_sim, meanlog = log(v1), sdlog = v2),
      "normal" = rnorm(n = model_params$n_sim, mean = v1, sd = v2),
      "gamma" = rgamma(n = model_params$n_sim, shape = v1, rate = 1/v2),
      "fixed" = rep(v1, model_params$n_sim),
      stop("Unsupported distribution type")
    )
    
    # Store the samples in the list with the parameter name
    df_list[[param_name]] <- samples
  }
  
  # Convert the list to a data frame
  df <- as.data.frame(df_list)
  return(df)
}

When we print out the head and dimensions of this PSA data frame, we can see that it contains 1000 rows and 12 columns. Each row corresponds to one of our generated patients, and their parameters all differ slightly.

In [5]:
df_psa <- f_gen_psa(default_psa_params, model_params)

print(paste("rows:", dim(df_psa)[1], "  columns:", dim(df_psa)[2]))
print(head(df_psa))

[1] "rows: 1000   columns: 12"


  p_S2_S3_unt  hr_S2_S3       p_S2_D    hr_S3_D     c_S2     c_S3 c_Trt_S2
1   0.1391695 0.7276875 5.268306e-05 0.11924909 1845.357 4032.251      200
2   0.1494417 0.9359672 1.477189e-04 0.11288899 2052.895 3819.168      200
3   0.1849718 0.6881810 2.289227e-05 0.06552596 1894.417 4096.676      200
4   0.1623579 0.7944042 1.245705e-03 0.11334684 2036.821 3876.154      200
5   0.1320931 0.2627625 3.503446e-04 0.08177929 2056.848 3845.596      200
6   0.1450146 0.3298773 2.064830e-04 0.07501609 1755.131 3983.651      200
  c_Trt_S3 c_D      u_S2      u_S3 u_D
1        0   0 1.0146819 0.7177388   0
2        0   0 0.9922415 0.7963175   0
3        0   0 0.9982299 0.7453033   0
4        0   0 1.0051137 0.7378315   0
5        0   0 0.9954517 0.7843815   0
6        0   0 1.0133173 0.7461986   0


Next, we calculate our transition probabilities for S2 to S3 when treated, and for S3 to dead compared with S2 to dead. These probabilities aren't directly set as parameters, rather, we set hazard ratios which themselves have a probability distribution. This allows us to account for uncertainty in parameters that are key to evaluating the treatment's performance.

In [6]:
# Calculate transition rates and probabilities
calculate_transition_rates <- function(psa_table_row) {
  with(as.list(psa_table_row), {
    r_S2_S3_unt <- -log(1 - p_S2_S3_unt)
    r_S2_S3_trt <- hr_S2_S3 * r_S2_S3_unt
    p_S2_S3_trt <- 1 - exp(-r_S2_S3_trt)
    
    r_S2_D <- -log(1 - p_S2_D)
    r_S3_D <- hr_S3_D * r_S2_D
    p_S3_D <- 1 - exp(-r_S3_D)
    
    list(r_S2_S3_unt = r_S2_S3_unt, r_S2_S3_trt = r_S2_S3_trt, p_S2_S3_trt = p_S2_S3_trt,
         r_S2_D = r_S2_D, r_S3_D = r_S3_D, p_S3_D = p_S3_D)
  })
}

In [7]:
# Extract parameters for the first row
psa_table_row <- df_psa[1,]

transition_rates <- calculate_transition_rates(psa_table_row)
print(transition_rates)

$r_S2_S3_unt
[1] 0.1498577

$r_S2_S3_trt
[1] 0.1090495

$p_S2_S3_trt
[1] 0.103314

$r_S2_D
[1] 5.268444e-05

$r_S3_D
[1] 6.282572e-06

$p_S3_D
[1] 6.282552e-06



Our model uses discount rates : this means that costs and utilities are considered to decrease in value with time, since it is preferable to have funds/utility now rather than later. The Haute Autorité de Santé (HAS) recommends a discount rate of 4%, which is what our example is set to. Based on that discount rate, we calculate year-over-year the cumulative discounting that has happened : these are our discount weights. 

In [9]:
calculate_discount_weights <- function(model_params) {
  n_t <- model_params$n_t
  d_r <- model_params$d_r
  v_dwe <- v_dwc <- 1 / (1 + d_r) ^ (0:n_t)
  list(v_dwe = v_dwe, v_dwc = v_dwc)
}

In [10]:
discount_weights <- calculate_discount_weights(model_params)
print(discount_weights)

$v_dwe
[1] 1.0000000 0.9615385 0.9245562 0.8889964 0.8548042 0.8219271 0.7903145
[8] 0.7599178

$v_dwc
[1] 1.0000000 0.9615385 0.9245562 0.8889964 0.8548042 0.8219271 0.7903145
[8] 0.7599178



This part of the code generates transitions matrices. These matrices simply show the transition probabilities from each state to each state. There is one for each of our Markov models : the one with treatment, and the one without it. Of course, the only differences are the probabilities related to staying in Stage 2, or transitioning to Stage 3. 

In [11]:
create_transition_matrices <- function(psa_table_row, transition_rates) {
  with(as.list(psa_table_row), {
    m_P_unt <- m_P_trt <- matrix(0, nrow = n_states, ncol = n_states, dimnames = list(v_n, v_n))
    
    # Untreated
    m_P_unt["S2", "S2"] <- 1 - (p_S2_S3_unt + p_S2_D)
    m_P_unt["S2", "S3"] <- p_S2_S3_unt
    m_P_unt["S2", "D"] <- p_S2_D
    m_P_unt["S3", "S3"] <- 1 - transition_rates$p_S3_D
    m_P_unt["S3", "D"] <- transition_rates$p_S3_D
    m_P_unt["D", "D"] <- 1
    
    # Treated
    m_P_trt["S2", "S2"] <- 1 - (transition_rates$p_S2_S3_trt + p_S2_D)
    m_P_trt["S2", "S3"] <- transition_rates$p_S2_S3_trt
    m_P_trt["S2", "D"] <- p_S2_D
    m_P_trt["S3", "S3"] <- 1 - transition_rates$p_S3_D
    m_P_trt["S3", "D"] <- transition_rates$p_S3_D
    m_P_trt["D", "D"] <- 1
    
    list(m_P_unt = m_P_unt, m_P_trt = m_P_trt)
  })
}

In [12]:
psa_table_row <- df_psa[1,]

transition_matrices <- create_transition_matrices(psa_table_row, transition_rates)
print(transition_matrices)

$m_P_unt
          S2        S3            D
S2 0.8607778 0.1391695 5.268306e-05
S3 0.0000000 0.9999937 6.282552e-06
D  0.0000000 0.0000000 1.000000e+00

$m_P_trt
          S2        S3            D
S2 0.8966333 0.1033140 5.268306e-05
S3 0.0000000 0.9999937 6.282552e-06
D  0.0000000 0.0000000 1.000000e+00



Now that we have our transition probability matrices, we can build the actual Markov model. Through the given range of ages that the model is to be run at, we calculate the proportion of individuals that are to be in each state, based on our transition probabilities. 

In [13]:
simulate_markov_traces <- function(model_params, transition_matrices) {
  n_t <- model_params$n_t
  m_TR_trt <- m_TR_unt <- matrix(NA, nrow = n_t + 1, ncol = n_states, dimnames = list(0:n_t, v_n))
  m_TR_unt[1, ] <- m_TR_trt[1, ] <- c(1, 0, 0)
  
  for (t in 1:n_t) {
    m_TR_unt[t + 1, ] <- m_TR_unt[t, ] %*% transition_matrices$m_P_unt
    m_TR_trt[t + 1, ] <- m_TR_trt[t, ] %*% transition_matrices$m_P_trt
  }
  
  list(m_TR_unt = m_TR_unt, m_TR_trt = m_TR_trt)
}

We can see here how the model runs for multiple iterations, until it reaches a final state, at the end of the given number of years.

I find this to be the most interesting part, the core of Markov modeling. These tables show us what proportion of individuals were at which state, each year.

In [14]:
markov_traces <- simulate_markov_traces(model_params, transition_matrices)
print(markov_traces)

$m_TR_unt
         S2        S3            D
0 1.0000000 0.0000000 0.000000e+00
1 0.8607778 0.1391695 5.268306e-05
2 0.7409385 0.2589626 9.890580e-05
3 0.6377834 0.3620770 1.395676e-04
4 0.5489898 0.4508348 1.754428e-04
5 0.4725582 0.5272346 2.071976e-04
6 0.4067676 0.5929969 2.354058e-04
7 0.3501366 0.6496029 2.605611e-04

$m_TR_trt
         S2        S3            D
0 1.0000000 0.0000000 0.000000e+00
1 0.8966333 0.1033140 5.268306e-05
2 0.8039513 0.1959481 1.005695e-04
3 0.7208495 0.2790063 1.441552e-04
4 0.6463377 0.3534784 1.838846e-04
5 0.5795279 0.4202519 2.201564e-04
6 0.5196240 0.4801227 2.533279e-04
7 0.4659122 0.5338041 2.837197e-04



The Markov trace is then used to calculate the costs and utilities, corresponding to the time spent in a state, multiplied by the cost and utility associated with the state. We also calculate the ICER.

In [15]:
calculate_costs_qalys <- function(psa_table_row, markov_traces, discount_weights) {
  with(as.list(psa_table_row), {
    v_u <- c(u_S2, u_S3, u_D)
    v_c_trt <- c(c_S2 + c_Trt_S2, c_S3, c_D)
    v_c_unt <- c(c_S2, c_S3, c_D)
    
    v_E_unt <- markov_traces$m_TR_unt %*% v_u
    v_E_trt <- markov_traces$m_TR_trt %*% v_u
    v_C_unt <- markov_traces$m_TR_unt %*% v_c_unt
    v_C_trt <- markov_traces$m_TR_trt %*% v_c_trt
    
    te_unt <- t(v_E_unt) %*% discount_weights$v_dwe
    te_trt <- t(v_E_trt) %*% discount_weights$v_dwe
    tc_unt <- t(v_C_unt) %*% discount_weights$v_dwc
    tc_trt <- t(v_C_trt) %*% discount_weights$v_dwc
    
    results <- c(
      "Cost_NoTrt" = tc_unt,
      "Cost_Trt" = tc_trt,
      "QALY_NoTrt" = te_unt,
      "QALY_Trt" = te_trt,
      "ICER" = (tc_trt - tc_unt) / (te_trt - te_unt)
    )
    return(results)
  })
}

In [16]:
psa_table_row <- df_psa[1,]
results <- calculate_costs_qalys(psa_table_row, markov_traces, discount_weights)
print(results)

  Cost_NoTrt     Cost_Trt   QALY_NoTrt     QALY_Trt         ICER 
18334.580290 18216.317505     6.368598     6.521450  -773.704858 


Now, all of the previously shown functions are in practice meant to be run all at once in our app, so we wrap them here.

In [9]:
f_hybrid_markov_model <- function(psa_table_row, model_params) {
  environment(create_transition_matrices) <- environment()
  environment(simulate_markov_traces)     <- environment()

  transition_rates <- calculate_transition_rates(psa_table_row)
  discount_weights <- calculate_discount_weights(model_params)
  transition_matrices <- create_transition_matrices(psa_table_row, transition_rates)
  markov_traces <- simulate_markov_traces(model_params, transition_matrices)
  results <- calculate_costs_qalys(psa_table_row, markov_traces, discount_weights)
  return(results)
}

A second wrapper function accesses the `psa_params` and `model_params` data frames, defines the necessary variables previously set in the environment, and runs our Markov model for the necessary number of times. It therefore outputs the costs and utilities for both treated and untreated groups, as well as the ICER, for each simulation performed. These data can later be represented on a plot, and their averages on a table.

In [16]:

f_wrapper <- function(psa_params, model_params) {
  # need to specify environment of inner functions (to use outer function enviroment)
  # alternatively - define functions within the wrapper function.
  environment(f_gen_psa)               <- environment()
  environment(f_hybrid_markov_model)   <- environment()

  df_psa <- f_gen_psa(psa_params, model_params)

  # the 4 health states of the model:
  v_n <- c("S2", "S3", "D") 
  # number of health states 
  n_states <- length(v_n) 

  n_sim <- model_params$n_sim

  # Initialize matrix of results outcomes
  m_out <- matrix(NaN, 
                  nrow = n_sim, 
                  ncol = 5,
                  dimnames = list(1:n_sim,
                                  c("Cost_NoTrt", "Cost_Trt",
                                    "QALY_NoTrt", "QALY_Trt",
                                    "ICER")))

  # run model for each row of PSA inputs
  for(i in 1:n_sim){

    # store results in row of results matrix
    m_out[i,] <- f_hybrid_markov_model(df_psa[i, ], model_params)

  } # close model loop


  #-- Return results --#

  # convert matrix to dataframe (for plots)
  df_out <- as.data.frame(m_out) 

  # output the dataframe from the function  
  return(df_out) 
}

df_out <- f_wrapper(default_psa_params, model_params)

In [17]:
print(head(df_out))

  Cost_NoTrt Cost_Trt QALY_NoTrt QALY_Trt       ICER
1   18356.83 18037.86   6.406952 6.538932  -2416.865
2   20203.63 20770.59   6.339922 6.388120  11763.195
3   21993.34 20407.81   6.154789 6.460931  -5179.078
4   19134.06 19212.13   6.189467 6.305177    674.669
5   22150.18 22967.13   6.451873 6.454089 368710.616
6   17324.08 16458.24   6.146497 6.391474  -3534.381
